In [5]:
import pandas as pd
import glob
import os

def load_weather_data(file_path, file_type):
    """
    기상청(KMA)에서 받은 날씨 데이터 파일을 로드하고 정리합니다.
    (수동 헤더 지정 방식)
    """
    skip_rows = 0
    date_col = ''
    cols_to_use = []

    # 1. 파일 유형에 따라 필요한 설정 정의
    if file_type == 'temp':
        skip_rows = 7  # 기온 파일은 8번째 줄부터 시작
        date_col = '날짜'
        cols_to_use = ['날짜', '평균기온(℃)', '최고기온(℃)']
    elif file_type == 'precip':
        skip_rows = 7  # 강수량 파일도 8번째 줄부터 시작
        date_col = '날짜'
        cols_to_use = ['날짜', '강수량(mm)']
    elif file_type == 'humidity':
        skip_rows = 16 # 습도 파일은 17번째 줄부터 시작
        date_col = '일시' # 습도 파일은 날짜 컬럼명이 '일시'임
        cols_to_use = ['일시', '평균습도(%rh)']

    try:
        # 2. CSV 로드 (헤더 없이, 앞부분 스킵)
        df = pd.read_csv(
            file_path,
            header=None,            # 헤더가 없는 것으로 간주
            skiprows=skip_rows,     # 메타데이터 줄 건너뛰기
            encoding='cp949',       # 'utf-8' -> 'cp949'로 변경
            sep=',',                # 쉼표(,)를 구분자로 명시
            skipinitialspace=True   # 구분자 뒤 공백 무시
        )

        # 3. 수동으로 헤더 설정
        new_header = df.iloc[0].astype(str).str.strip()
        df.columns = new_header

        # 4. 데이터에서 헤더 줄(첫 번째 줄) 제거
        df = df.iloc[1:].reset_index(drop=True)

        # 5. 필요한 컬럼만 선택
        df = df[cols_to_use]

        # 6. 날짜 컬럼명 표준화 ('일시' -> '날짜')
        if date_col == '일시':
            df = df.rename(columns={'일시': '날짜'})

        # 7. 날짜 컬럼 정리 및 변환
        df['날짜'] = df['날짜'].astype(str).str.strip()
        df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce')
        df = df.dropna(subset=['날짜']) # 날짜 변환 실패한 행 제거

        # 8. '날짜'를 인덱스로 설정
        df = df.set_index('날짜')

        # 9. 강수량 데이터 특별 처리
        if file_type == 'precip':
            df['강수량(mm)'] = df['강수량(mm)'].fillna(0)
            df['강수량(mm)'] = pd.to_numeric(df['강수량(mm)'], errors='coerce')

        # 10. (안전장치) 다른 모든 컬럼도 숫자형으로 변환
        for col in df.columns:
            if col != '날짜':
                df[col] = pd.to_numeric(df[col], errors='coerce')

        print(f"Loaded {file_path} successfully.")
        return df

    except Exception as e:
        print(f"{file_path} 파일 처리 중 오류 발생: {e}")
        return pd.DataFrame() # 오류 시 빈 DataFrame 반환

# --- 메인 실행 코드 ---

# 1. glob을 사용해 파일 목록 자동으로 찾기
print("파일 검색 중...")
temp_files = sorted(glob.glob('temperature_*.csv'))
precip_files = sorted(glob.glob('precipitation_*.csv'))
humidity_files = sorted(glob.glob('humidity_*.csv'))

# 2. 모든 파일을 (파일 유형, 파일 경로) 튜플 리스트로 결합
files_to_process = []
files_to_process.extend([('temp', f) for f in temp_files])
files_to_process.extend([('precip', f) for f in precip_files])
files_to_process.extend([('humidity', f) for f in humidity_files])

print(f"총 {len(files_to_process)}개의 파일을 처리합니다.")

# 3. 컬럼명 단순화를 위한 딕셔너리
rename_map = {
    '평균기온(℃)': '평균기온',
    '최고기온(℃)': '최고기온',
    '강수량(mm)': '강수량',
    '평균습도(%rh)': '평균습도'
}

# 4. 개별 파일 처리 루프
output_files = []
for file_type, file_path in files_to_process:
    print(f"\n--- {file_path} 처리 시작 ---")

    # 5. 위에서 만든 함수로 데이터 로드 및 정리
    df = load_weather_data(file_path, file_type)

    # 6. 로드 성공 시 파일 저장
    if not df.empty:
        # 7. 컬럼명 단순화
        df = df.rename(columns=rename_map)

        # 8. 새 파일명 생성 (예: 'cleaned_temperature_1908-1910.csv')
        # os.path.basename()은 'temperature_1908-1910.csv' 부분만 추출
        output_filename = f"cleaned_{os.path.basename(file_path)}"

        # 9. 새 CSV 파일로 저장
        df.to_csv(output_filename)

        print(f"--- 성공: {output_filename} 으로 저장 ---")
        output_files.append(output_filename)
    else:
        print(f"--- 실패: {file_path} 파일 처리 실패 또는 빈 파일 ---")

print(f"\n\n총 {len(output_files)}개의 파일 처리가 완료되었습니다.")
print("생성된 파일 목록:")
for f in output_files:
    print(f)

파일 검색 중...
총 9개의 파일을 처리합니다.

--- temperature_1908-1910.csv 처리 시작 ---
Loaded temperature_1908-1910.csv successfully.
--- 성공: cleaned_temperature_1908-1910.csv 으로 저장 ---

--- temperature_2003-2010.csv 처리 시작 ---
Loaded temperature_2003-2010.csv successfully.
--- 성공: cleaned_temperature_2003-2010.csv 으로 저장 ---

--- temperature_2305-2310.csv 처리 시작 ---
Loaded temperature_2305-2310.csv successfully.
--- 성공: cleaned_temperature_2305-2310.csv 으로 저장 ---

--- precipitation_1908-1910.csv 처리 시작 ---
Loaded precipitation_1908-1910.csv successfully.
--- 성공: cleaned_precipitation_1908-1910.csv 으로 저장 ---

--- precipitation_2003-2010.csv 처리 시작 ---
Loaded precipitation_2003-2010.csv successfully.
--- 성공: cleaned_precipitation_2003-2010.csv 으로 저장 ---

--- precipitation_2305-2310.csv 처리 시작 ---
Loaded precipitation_2305-2310.csv successfully.
--- 성공: cleaned_precipitation_2305-2310.csv 으로 저장 ---

--- humidity_1908-1910.csv 처리 시작 ---
Loaded humidity_1908-1910.csv successfully.
--- 성공: cleaned_humidity_1908-19

In [6]:
import pandas as pd
import glob
import os

# 1. 이전 단계에서 생성된 'cleaned_' 파일들이 있는지 확인합니다.
cleaned_files = sorted(glob.glob('cleaned_*.csv'))
if len(cleaned_files) < 9:
    print(f"오류: 'cleaned_' 파일이 9개 미만입니다 (현재 {len(cleaned_files)}개).")
    print("이전 스크립트를 먼저 실행하여 9개의 정리된 파일을 생성해야 합니다.")
else:
    print(f"총 {len(cleaned_files)}개의 정리된 파일을 찾았습니다. 병합 작업을 시작합니다.")

    # 2. 병합할 기준이 되는 년도(기간) 접미사를 정의합니다.
    suffixes = ['1908-1910', '2003-2010', '2305-2310']
    output_files_list = []

    # 3. 각 접미사(기간)별로 루프를 돕니다.
    for suffix in suffixes:
        print(f"\n--- '{suffix}' 기간 그룹 처리 중 ---")

        # 4. 해당 기간의 파일 3개 경로를 정의합니다.
        temp_file = f"cleaned_temperature_{suffix}.csv"
        precip_file = f"cleaned_precipitation_{suffix}.csv"
        humid_file = f"cleaned_humidity_{suffix}.csv"

        try:
            # 5. 3개의 파일을 로드합니다.
            # '날짜' 컬럼을 인덱스로 바로 불러오고, 날짜 형식으로 파싱합니다.
            df_temp = pd.read_csv(temp_file, index_col='날짜', parse_dates=True)
            df_precip = pd.read_csv(precip_file, index_col='날짜', parse_dates=True)
            df_humid = pd.read_csv(humid_file, index_col='날짜', parse_dates=True)

            print(f"로드 완료: {temp_file}, {precip_file}, {humid_file}")

            # 6. '날짜' 인덱스를 기준으로 3개의 데이터프레임을 하나로 합칩니다(join).
            # how='outer'는 한쪽에만 데이터가 있어도 날짜를 유지하는 안전한 방식입니다.
            df_combined = df_temp.join(df_precip, how='outer')
            df_combined = df_combined.join(df_humid, how='outer')

            # 7. 최종적으로 날짜순으로 정렬합니다.
            df_combined = df_combined.sort_index()

            # 8. 저장할 최종 파일명을 정의합니다. (예: weather_combined_1908-1910.csv)
            output_filename = f"weather_combined_{suffix}.csv"

            # 9. 새 CSV 파일로 저장합니다.
            df_combined.to_csv(output_filename)

            print(f"--- 성공: {output_filename} 파일로 저장되었습니다. ---")
            output_files_list.append(output_filename)

        except FileNotFoundError as e:
            print(f"오류: {suffix} 그룹의 파일을 찾을 수 없습니다. 누락된 파일: {e.filename}")
        except Exception as e:
            print(f"'{suffix}' 그룹 처리 중 예기치 않은 오류 발생: {e}")

    print("\n\n모든 병합 작업이 완료되었습니다.")
    print("최종 생성된 파일 3개:")
    for f in output_files_list:
        print(f)

총 9개의 정리된 파일을 찾았습니다. 병합 작업을 시작합니다.

--- '1908-1910' 기간 그룹 처리 중 ---
로드 완료: cleaned_temperature_1908-1910.csv, cleaned_precipitation_1908-1910.csv, cleaned_humidity_1908-1910.csv
--- 성공: weather_combined_1908-1910.csv 파일로 저장되었습니다. ---

--- '2003-2010' 기간 그룹 처리 중 ---
로드 완료: cleaned_temperature_2003-2010.csv, cleaned_precipitation_2003-2010.csv, cleaned_humidity_2003-2010.csv
--- 성공: weather_combined_2003-2010.csv 파일로 저장되었습니다. ---

--- '2305-2310' 기간 그룹 처리 중 ---
로드 완료: cleaned_temperature_2305-2310.csv, cleaned_precipitation_2305-2310.csv, cleaned_humidity_2305-2310.csv
--- 성공: weather_combined_2305-2310.csv 파일로 저장되었습니다. ---


모든 병합 작업이 완료되었습니다.
최종 생성된 파일 3개:
weather_combined_1908-1910.csv
weather_combined_2003-2010.csv
weather_combined_2305-2310.csv
